In [ ]:
#| default_exp handlers.data_format_transformation

# Data format transformation  

> A data pipeline handler that transforms MARIS data from NetCDF to CSV. The primary focus is on converting NetCDF data into MARIS Standard Open-Refine format while preserving data integrity. This handler implements a modular transformation pipeline using callbacks for each processing step, ensuring flexibility and extensibility in data handling.

:::{.callout-tip}

For new MARIS users, please refer to [field definitions
](https://github.com/franckalbinet/marisco/blob/main/nbs/metadata/field-definition.ipynb) for detailed information about Maris fields.

:::

# Dependencies
> Required packages and internal modules for data format transformations

In [ ]:
#| export
from pathlib import Path
from netCDF4 import Dataset
import pandas as pd
import fastcore.all as fc
from typing import Dict, Callable

from marisco.configs import (
    NC_VARS,
    OR_VARS,
    NC_GROUPS,
    OR_DTYPES,
    Enums,
    lut_path,
    species_lut_path,
    cfg
)

from marisco.utils import (
    get_netcdf_properties
)

from marisco.callbacks import (
    Callback,
    Transformer,
    DecodeTimeCB,
    AddSampleTypeIdColumnCB
)  
    
from marisco.decoders import (
        NetCDFDecoder
    )
from marisco.metadata import (
    ZoteroItem
)

/tmp/ipykernel_53037/2716152763.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Configuration and File Paths

In [ ]:
#| eval: false
fname_in =  Path('../../_data/output/100-HELCOM-MORS-2024.nc')
fname_out = fname_in.with_suffix('.csv')
output_format = 'openrefine_csv'

## Data Loading

Load data from standardized MARIS NetCDF files. The NetCDF files follow CF conventions and include standardized variable names and metadata according to MARIS specifications.

In [ ]:
#| exports
def load_data(fname:str, verbose: bool = False):
    """Load NetCDF groups into DataFrames with standardized column names."""
    dfs = {}
    with Dataset(fname, 'r') as nc:
        for group_name in nc.groups:
            group = nc.groups[group_name]
            # Get all variables in the group
            data = {var_name: var[:] for var_name, var in group.variables.items() if var_name not in group.dimensions}
            # Convert to DataFrame
            df = pd.DataFrame(data)
            # Rename columns using NC_VARS mapping
            rename_map = {nc_var: col for col, nc_var in NC_VARS.items() 
                         if nc_var in df.columns}
            # here update to infrom if df.columns includes names that are not included in NC_VARS. Just print 
            
            
            df = df.rename(columns=rename_map)
            dfs[group_name.upper()] = df
            if verbose:
                print(f"Loaded group {group_name} with columns: {df.columns.tolist()}")
        
        # herer extract enum_dicts 
        
        #here get global attibutes 
        
        
    
    return dfs, enum_dicts, globattrs

In [ ]:
#|eval: false
dfs = load_to_dataframes(fname_in, verbose=True)

Loaded group biota with columns: ['LON', 'LAT', 'SMP_DEPTH', 'TIME', 'NUCLIDE', 'VALUE', 'UNIT', 'UNC', 'DL', 'BIO_GROUP', 'SPECIES', 'BODY_PART', 'DRYWT', 'WETWT', 'PERCENTWT']
Loaded group seawater with columns: ['LON', 'LAT', 'SMP_DEPTH', 'TOT_DEPTH', 'TIME', 'NUCLIDE', 'VALUE', 'UNIT', 'UNC', 'DL', 'FILT']
Loaded group sediment with columns: ['LON', 'LAT', 'TOT_DEPTH', 'TIME', 'NUCLIDE', 'VALUE', 'UNIT', 'UNC', 'DL', 'SED_TYPE', 'TOP', 'BOTTOM', 'PERCENTWT']


## Validate NetCDF Enumerations

Verify that enumerated values in the NetCDF file match current MARIS lookup tables.

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: The enumeration validation process is a diagnostic step that identifies inconsistencies between NetCDF enumerations and MARIS lookup tables. While this validation does not modify the dataset, it generates detailed feedback about any mismatches or undefined values. 


:::

In [ ]:
#| exports
class ValidateEnumsCB(Callback):
    "Validate enumeration mappings between NetCDF file and MARIS lookup tables."
    def __init__(self, 
                src_fname: str,  # Path to NetCDF file
                enums: Enums,    # MARIS lookup table enums
                verbose: bool = False
                ):
        fc.store_attr()
        
    def __call__(self, tfm: Transformer):
        """Process each group in the NetCDF file and validate its enums."""
        with Dataset(self.src_fname, 'r') as nc:
            for group_name in nc.groups:
                group = nc.groups[group_name]
                self._validate_group(group, group_name)
    
    def _validate_group(self, group, group_name: str):
        """Validate enum mappings for a specific group."""
        for var_name, var in group.variables.items():
            if not hasattr(var.datatype, 'enum_dict'): 
                continue
            
            nc_enum_dict = var.datatype.enum_dict
            if self.verbose:
                print(f"nc_enum_dict [{var_name}]:", nc_enum_dict)

            # Get original column name from NC_VARS mapping
            original_col = next((col for col, nc_var in NC_VARS.items() 
                               if nc_var == var_name), None)
            if not original_col: 
                continue

            # Compare enum mappings
            self._compare_mappings(
                nc_enum_dict,
                self.enums.types[original_col],
                group_name,
                var_name,
                original_col
            )
    
    def _compare_mappings(self, nc_dict: dict, lut_dict: dict, 
                         group_name: str, var_name: str, col_name: str):
        """Compare NetCDF enum dictionary with lookup table dictionary."""
        if self.verbose:
            print(f"lut_enum [{col_name}]:", lut_dict)
            
        # Check for mismatches between NetCDF and lookup table
        for key, value in nc_dict.items():
            if key not in lut_dict or lut_dict[key] != value:
                print(f"\nWarning: Enum mismatch in {group_name}/{var_name}")
                print(f"NetCDF value: {key} -> {value}")
                print(f"Lookup value: {key} -> {lut_dict.get(key, 'Not found')}")        

In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
tfm = Transformer(
    dfs,
    cbs=[
        ValidateEnumsCB(
            src_fname=fname_in,
            enums=Enums(lut_src_dir=lut_path()),
            #verbose=True
        ),
    ]
)
tfm()
print('\n')

## Remove Non Open Refine Columns 

The ``RemoveNonORVarsCB`` class filters out variables from the NetCDF format that do not align with the requirements for MARIS's OpenRefine data processing.

In [ ]:
#| exports
class RemoveNonORVarsCB(Callback):
    "Remove variables not defined in OR_VARS configuration."
    def __init__(self, 
                or_vars: Dict[str, str] = OR_VARS,  # Dictionary mapping OR vars to NC vars
                verbose: bool = False,
                ):
        fc.store_attr()
        
    def __call__(self, tfm: Transformer):
        """Remove non-OR variables from all dataframes."""
        for group_name in tfm.dfs:
            tfm.dfs[group_name] = self._remove_non_or_vars(tfm.dfs[group_name], group_name)
            
    def _remove_non_or_vars(self, df: pd.DataFrame, group_name:str ) -> pd.DataFrame:
        """Remove columns not in OR_VARS and print removed columns if verbose."""
        current_cols = set(df.columns)
        or_cols = set(self.or_vars.keys())
        cols_to_remove = current_cols - or_cols
        
        if self.verbose and cols_to_remove:
            print(f"Removing variables that are not compatible with MARIS's OpenRefine processing. \nRemoving {', '.join(cols_to_remove)} from {group_name} dataset.")
                        
        return df.drop(columns=cols_to_remove)


In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
tfm = Transformer(
    dfs,
    cbs=[
        RemoveNonORVarsCB(verbose=True),
    ]
)
tfm()
print('\n')

Removing variables that are not compatible with MARIS's OpenRefine processing. 
Removing BIO_GROUP from BIOTA dataset.




## Add Taxon Information

In [ ]:
#| exports
TAXON_KEY_MAP = {
    'Taxonname': 'TAXONNAME',
    'Taxonrank': 'TAXONRANK',
    'TaxonDB': 'TAXONDB',
    'TaxonDBID': 'TAXONDBID',
    'TaxonDBURL': 'TAXONDBURL'
}

In [ ]:
#| exports
def get_taxon_info_lut(maris_lut: str, key_names: dict = TAXON_KEY_MAP) -> dict:
    "Create lookup dictionary for taxon information from MARIS species lookup table."
    species = pd.read_excel(maris_lut)
    # Select columns and rename them to standardized format
    columns = ['species_id'] + list(key_names.keys())
    df = species[columns].rename(columns=key_names)
    return df.set_index('species_id').to_dict()

lut_taxon = lambda: get_taxon_info_lut(maris_lut=species_lut_path(), key_names=TAXON_KEY_MAP)

In [ ]:
#| exports
class AddTaxonInformationCB(Callback):
    """Add taxon information to BIOTA group based on species lookup table."""
    
    def __init__(self, 
                fn_lut: Callable = lut_taxon,  # Function that returns taxon lookup dictionary
                verbose: bool = False
                ):
        fc.store_attr()
        
    def __call__(self, tfm: Transformer):
        """Delegate tasks to add taxon information to the BIOTA group."""
        if not self.check_biota_group_exists(tfm):
            return
        
        df = tfm.dfs['BIOTA']
        if not self.check_species_column_exists(df):
            return
        
        self.add_taxon_columns(df)

    def check_biota_group_exists(self, tfm: Transformer) -> bool:
        """Check if 'BIOTA' group exists in the dataframes."""
        if 'BIOTA' not in tfm.dfs:
            if self.verbose:
                print("No BIOTA group found, skipping taxon information")
            return False
        return True

    def check_species_column_exists(self, df: pd.DataFrame) -> bool:
        """Check if 'SPECIES' column exists in the BIOTA dataframe."""
        if 'SPECIES' not in df.columns:
            if self.verbose:
                print("No SPECIES column found in BIOTA dataframe, skipping taxon information")
            return False
        return True

    def add_taxon_columns(self, df: pd.DataFrame):
        """Add taxon information columns to the BIOTA dataframe."""
        lut = self.fn_lut()
        
        # Add each column from the lookup table
        for col in lut.keys():
            df[col] = df['SPECIES'].map(lut[col]).fillna('Unknown')
        
        self.report_unmatched_species(df)

    def report_unmatched_species(self, df: pd.DataFrame):
        """Report any species IDs not found in the lookup table."""
        unmatched = df[df['TAXONNAME'] == 'Unknown']['SPECIES'].unique()
        if self.verbose and len(unmatched) > 0:
            print(f"Warning: Species IDs not found in lookup table: {', '.join(map(str, unmatched))}")

In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
tfm = Transformer(
    dfs,
    cbs=[
        AddTaxonInformationCB(
            fn_lut=lut_taxon
        ),
    ]
)

tfm()
print(tfm.dfs['BIOTA'][['TAXONNAME','TAXONRANK','TAXONDB','TAXONDBID','TAXONDBURL']])


               TAXONNAME TAXONRANK   TAXONDB TAXONDBID  \
0           Gadus morhua   species  Wikidata   Q199788   
1           Gadus morhua   species  Wikidata   Q199788   
2           Gadus morhua   species  Wikidata   Q199788   
3           Gadus morhua   species  Wikidata   Q199788   
4           Gadus morhua   species  Wikidata   Q199788   
...                  ...       ...       ...       ...   
16089  Fucus vesiculosus   species  Wikidata   Q754755   
16090  Fucus vesiculosus   species  Wikidata   Q754755   
16091     Mytilus edulis   species  Wikidata    Q27855   
16092     Mytilus edulis   species  Wikidata    Q27855   
16093     Mytilus edulis   species  Wikidata    Q27855   

                                  TAXONDBURL  
0      https://www.wikidata.org/wiki/Q199788  
1      https://www.wikidata.org/wiki/Q199788  
2      https://www.wikidata.org/wiki/Q199788  
3      https://www.wikidata.org/wiki/Q199788  
4      https://www.wikidata.org/wiki/Q199788  
...                  

## Remap to OR mappings

> **Note:** This operation must take place before `ConvertToHumanReadableCB` as it relies on the data being in its encoded form for accurate mapping.

In [ ]:
#| exports
or_mappings={'DL':
                {0:'ND',1:'=',2:'<'},
            'FILT':
                {0:'NA',1:'Y',2:'N'},
            }

In [ ]:
#| exports
class RemapToORMappingsCB(Callback):
    "Convert values using OR mappings if columns exist in dataframe."
    def __init__(self, 
                or_mappings: Dict[str, Dict] = or_mappings,  # Dictionary of column mappings, 
                verbose: bool = False
                ):
        fc.store_attr()
        
    def __call__(self, tfm: Transformer):
        """Apply OR mappings to all dataframes."""
        for group_name in tfm.dfs:
            if self.verbose:
                print(f"\nProcessing {group_name} group...")
            tfm.dfs[group_name] = self._apply_mappings(tfm.dfs[group_name])
            
    def _apply_mappings(self, df: pd.DataFrame) -> pd.DataFrame:
        """Apply OR mappings to columns that exist in the dataframe."""
        for col, mapping in self.or_mappings.items():
            if col in df.columns:
                if self.verbose:
                    print(f"    Mapping values for column: {col}")
                df[col] = df[col].map(mapping)
        return df


In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
tfm = Transformer(
    dfs,
    cbs=[
        RemapToORMappingsCB(),
    ]
)

tfm()

# Loop through each group in the 'dfs' dictionary
for group_name, df in tfm.dfs.items():
    # Check if the group dataframe contains any of the columns specified in or_mappings.keys()
    relevant_columns = [col for col in or_mappings.keys() if col in df.columns]
    if relevant_columns:
        # Print the unique values from the relevant columns
        print(f"\nUnique values in {group_name} for columns {relevant_columns}:")
        for col in relevant_columns:
            print(f"{col}: {df[col].unique()}")
    else:
        print(f"No relevant columns found in {group_name} based on or_mappings keys.")


Unique values in BIOTA for columns ['DL']:
DL: ['<' '=' 'ND']

Unique values in SEAWATER for columns ['DL', 'FILT']:
DL: ['=' '<' 'ND']
FILT: ['NA' 'N' 'Y']

Unique values in SEDIMENT for columns ['DL']:
DL: ['=' '<' 'ND']


## Remap to human readable 

In [ ]:
OR_DTYPES

{'LON': {'type': 'human_readable'},
 'LAT': {'type': 'human_readable'},
 'SMP_DEPTH': {'type': 'human_readable'},
 'TOT_DEPTH': {'type': 'human_readable'},
 'TIME': {'type': 'human_readable'},
 'AREA': {'type': 'human_readable'},
 'NUCLIDE': {'type': 'encoded'},
 'VALUE': {'type': 'human_readable'},
 'UNIT': {'type': 'encoded'},
 'UNC': {'type': 'human_readable'},
 'DL': {'type': 'human_readable'},
 'FILT': {'type': 'human_readable'},
 'COUNT_MET': {'type': 'encoded'},
 'SAMP_MET': {'type': 'encoded'},
 'PREP_MET': {'type': 'encoded'},
 'VOL': {'type': 'human_readable'},
 'SAL': {'type': 'human_readable'},
 'TEMP': {'type': 'human_readable'},
 'SPECIES': {'type': 'encoded'},
 'BODY_PART': {'type': 'encoded'},
 'SED_TYPE': {'type': 'encoded'},
 'TOP': {'type': 'human_readable'},
 'BOTTOM': {'type': 'human_readable'},
 'DRYWT': {'type': 'human_readable'},
 'WETWT': {'type': 'human_readable'},
 'PERCENTWT': {'type': 'human_readable'},
 'LAB': {'type': 'encoded'},
 'PROFILE_ID': {'type': '

In [ ]:
#| exports
class RemapToHumanReadableCB(Callback):
    """Convert enum values in DataFrames to their human-readable format, but only for variables defined as 'human_readable' in OR_DTYPES and not present in or_mappings."""
    
    def __init__(self, 
                 or_dtypes: Dict = OR_DTYPES,  # Dictionary defining variable types
                 or_mappings: Dict = or_mappings,  # Dictionary of value mappings
                 output_format: str = 'openrefine_csv',
                 verbose: bool = False
                ):
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        """Delegate the conversion of numeric enum values to human-readable strings for specified variables."""
        if self.output_format != 'openrefine_csv':
            self.or_mappings = {}
        
        for group_name, df in tfm.dfs.items():
            self.process_group(group_name, df)

    def process_group(self, group_name, df):
        """Process each group to convert enums to human-readable format."""
        if self.verbose:
            print(f'Processing {group_name} enums ...')
        
        self.convert_variables(group_name, df)

    def convert_variables(self, group_name, df):
        """Convert variables within a group to their human-readable format if applicable."""
        for col_name in df.columns:
            if col_name in self.or_dtypes and self.or_dtypes[col_name]['type'] == 'human_readable':
                self.convert_variable(col_name, group_name, df)

    def convert_variable(self, col_name, group_name, df):
        """Convert a single variable to human-readable format based on conditions."""
        if col_name not in self.or_mappings and self.output_format == 'openrefine_csv':
            if self.verbose:
                print(f"Converting '{col_name}' to human readable format")
            
            # Assuming enum_dict is somehow accessible here, possibly through a modified approach or additional data structure
            enum_dict = self.get_enum_dict(col_name)
            df[col_name] = df[col_name].map(enum_dict).fillna('Unknown')
            
            if self.verbose:
                print(f"Converted {col_name} in {group_name}")
                print("-" * 80)
                
            
    def get_enum_dict(self, col_name):
        """Retrieve or simulate the enum dictionary for a column."""
        # Placeholder for actual enum dictionary retrieval logic
        return {1: 'Type A', 2: 'Type B', 3: 'Type C'}  # Example mapping

In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
output_format = 'csv' #'openrefine_csv'
tfm = Transformer(
    dfs,
    cbs=[
        RemoveNonORVarsCB(),
        RemapToHumanReadableCB(
            output_format=output_format,
            verbose=True
        ),
    ]
)
tfm()
print('\n')

Processing BIOTA enums ...
Processing SEAWATER enums ...
Processing SEDIMENT enums ...




## Standardize Time

In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
tfm = Transformer(
    dfs,
    cbs=[
        DecodeTimeCB(),
    ]
)

tfm()

print(tfm.dfs['BIOTA']['TIME'])


0       2012-09-23
1       2012-09-23
2       2012-09-23
3       2012-09-23
4       2012-09-23
           ...    
16089   2022-05-10
16090   2022-05-10
16091   2022-09-15
16092   2022-09-15
16093   2022-09-15
Name: TIME, Length: 16094, dtype: datetime64[ns]


## Add Sample Type ID

In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
tfm = Transformer(
    dfs,
    cbs=[
        AddSampleTypeIdColumnCB(),
    ]
)

tfm()
print(tfm.dfs['SEAWATER']['samptype_id'].unique())
print(tfm.dfs['BIOTA']['samptype_id'].unique())
print(tfm.dfs['SEDIMENT']['samptype_id'].unique())


[1]
[2]
[3]


## Add Reference ID


Include the `ref_id` (i.e., Zotero Archive Location) of the Maris data. The `ZoteroArchiveLocationCB` performs a lookup of the Zotero Archive Location based on the `Zotero key` defined in the global attributes of the MARIS NetCDF file as `id`.

In [ ]:

#| export
class AddZoteroArchiveLocationCB(Callback):
    "Fetch and append 'Loc. in Archive' from Zotero to DataFrame."
    def __init__(self, src_fname: str, cfg: dict):
        self.src_fname = src_fname
        self.cfg = cfg

    def __call__(self, tfm):
        
        zotero_key = get_netcdf_properties(self.src_fname)['global_attributes']['id']
        item = ZoteroItem(zotero_key, self.cfg['zotero'])
        if item.exist():
            loc_in_archive = item.item['data']['archiveLocation'] 
            for grp, df in tfm.dfs.items():
                df['REF_ID'] = int(loc_in_archive)
        else:
            print(f"Warning: Zotero item {self.item_id} does not exist.")

In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
tfm = Transformer(
    dfs,
    cbs=[
        AddZoteroArchiveLocationCB(src_fname=fname_in, cfg=cfg()),
    ]
)
tfm()
print(tfm.dfs['SEAWATER']['REF_ID'].unique())


[100]


## Review all callbacks

In [ ]:
#| eval: false
dfs = load_to_dataframes(fname_in)
output_format = 'csv' #'openrefine_csv' # 'csv'

cbs_validation=[ValidateEnumsCB(
            src_fname=fname_in,
            enums=Enums(lut_src_dir=lut_path())
            ),
        ValidateNetCDFVarsCB(
            src_fname=fname_in
            )]

cbs_or=[RemoveNonORVarsCB(),
            RemapToORMappingsCB(
            or_mappings=or_mappings,
            ),
            
            ]


cbs_general=[AddTaxonInformationCB(
            fn_lut=lut_taxon
            ),            
        RemapToHumanReadableCB(
            src_fname=fname_in, 
            output_format=output_format
            ),
        DecodeTimeCB(),
        AddSampleTypeIdColumnCB(),
        AddZoteroArchiveLocationCB(src_fname=fname_in, cfg=cfg())
    ]


            
            
         
         
         ]


tfm = Transformer(dfs,cbs)  
tfm()
print(tfm.dfs['BIOTA'])

NameError: name 'ValidateEnumsCB' is not defined

In [ ]:
tfm.dfs['SEAWATER']['DL'].unique()

array([nan], dtype=object)

## Decoding NETCDF

In [ ]:
#| export
def decode(
    fname_in: str, # Input file name
    dest_out: str | None = None, # Output file name (optional)
    output_format: str = 'openrefine_csv',
    remap_vars: Dict[str, str] = OR_VARS,
    verbose: bool = False,
    **kwargs # Additional arguments
    ) -> None:
    "Decode data from NetCDF."
    dfs = load_to_dataframes(fname_in)

    valid_output_formats=['openrefine_csv', 'csv']
    if output_format not in valid_output_formats:
        print (f'Invalid output format. Allowed formats: {valid_output_formats}')
        return 
    
    tfm = Transformer(
        dfs,
        cbs=[
            RemoveNonORVarsCB(
                output_format=output_format
                ),
            ValidateEnumsCB(
                src_fname=fname_in,
                enums=Enums(lut_src_dir=lut_path())
                ),
            ValidateNetCDFVarsCB(
                src_fname=fname_in
                ),
            
            AddTaxonInformationCB(
                fn_lut=lut_taxon
                ),  
            RemapToORMappingsCB(
                or_mappings=or_mappings,
                output_format=output_format
                ),            
            RemapToHumanReadableCB(
                src_fname=fname_in, 
                output_format=output_format
                ),
            DecodeTimeCB(),
            AddSampleTypeIdColumnCB(),
            AddZoteroArchiveLocationCB(src_fname=fname_in, cfg=cfg())
        ]
    )    
    
    tfm()
    decoder = NetCDFDecoder( 
                            dfs=tfm.dfs,
                            fname_in=fname_in,  
                            dest_out=dest_out,                           
                            output_format='csv',
                            remap_vars=OR_VARS,
                            verbose=verbose
                    )
    decoder.decode()

In [ ]:
#|eval: false
fname = Path('../../_data/output/100-HELCOM-MORS-2024.nc')
decode(fname_in=fname, dest_out=fname.with_suffix(''))

In [ ]:
decode(fname_in=fname, dest_out=fname.with_suffix(''))